# **ALPHA VOLATILITY GENERATION - SYSTEMATIC TRADING STRATEGIES PROJECT**

In [1]:
import numpy as np
from scipy.stats import norm
import pandas as pd

In [ ]:
class Market_data:

    def __init__(self, df_train, df_validation, df_test, df_price):
        self.df_train = df_train.copy()
        self.df_validation = df_validation.copy()
        self.df_test = df_test.copy()
        self.df_price = df_price.copy()

    
    def get_dataset(self, name):
        if name == "train":
            return self.df_train
        elif name == "validation":
            return self.df_validation
        elif name == "test":
            return self.df_test
        else:
            raise ValueError(f"Unknown dataset: {name}")


    def get_price_df(self):
        return self.df_price
    

    def get_row_date(self, date):
        ''' This function retrieves from the df_price DataFrame the row corresponding 
        to the date provided as a parameter.

        RETURNS
        -------
        type : pandas.core.series.Series
        '''
        rows = self.df_price[self.df_price["Date"] == date]
        if rows.empty:
            raise KeyError(f"Date {date} not found")
        return rows.iloc[0]


    def get_rows_date(self,date1,date2):
        '''This function retrieves from the DataFrame the spot prices for the 
        dates between date1 and date2.
        
        RETURNS
        -------
        type : pandas.core.frame.DataFrame
        '''
        filtered = self.df_price[(self.df_price['Date'] >= date1) & (self.df_price['Date'] <= date2)]
        return filtered

    def get_prices(self,date1,date2):
        '''This function returns the list of spot prices for dates between date1 and date2.

        RETURNS
        -------
        type : np.array
        '''
        filtered = self.get_rows_date(date1,date2)
        return filtered['Price'].to_numpy()


    def get_realized_volatility(self,prices):
        '''This function calculates the annualized realized volatility for a 
        given list of stock prices provided as the parameter prices.

        RETURNS
        -------
        type : float
        '''
        log_returns = np.diff(np.log(prices))
        realized_vol = np.sqrt(np.mean(log_returns ** 2) * 252)
        return realized_vol


    

In [ ]:
class Backtester:

    def __init__(self, market_data, strategy):
        '''
        PARAMETERS
        ----------
        market_data
            Class Market_data
        strategy
            Class Strategy
        '''
        self.market_data = market_data
        self.strategy = strategy
    
    # ------------------------- PRICERS AND GREEKS --------------------------------------
    def black_scholes_call_price(self, S, K, T, r, sigma):
        if T <= 0:
            return max(S - K, 0)
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2)* T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        return call_price

    def black_scholes_put_price(self, S, K, T, r, sigma):
        if T <= 0:
            return max(K - S, 0)
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        put_price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return put_price

    def black_scholes_call_delta(self, S, K, T, r, sigma):
        if T <= 0:
            return 1.0 if S > K else 0.0
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        return norm.cdf(d1)

    def black_scholes_put_delta(self, S, K, T, r, sigma):
        if T <= 0:
            return -1.0 if S < K else 0.0
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        return norm.cdf(d1) - 1 
    # -------------------------------------------------------------------------------

    def run_gamma_scalping(self, straddle_row):
        '''

        PARAMETERS
        ----------
        straddle_row : 
            pandas.core.series.Series
        '''

In [64]:
data = Market_data(df_train,df_validation,df_test,df_price)
data.get_prices(" 2020-07-25", " 2020-08-31")

array([ 94.81,  93.25,  95.04,  96.19, 106.26, 108.94, 109.67, 110.06,
       113.9 , 111.11, 112.73, 109.38, 113.01, 115.01, 114.91, 114.61,
       115.56, 115.71, 118.28, 124.37, 125.86, 124.83, 126.52, 125.01,
       124.81, 129.04])

In [65]:
data.get_rows_date(" 2020-07-25", " 2020-08-31")

,Date,Price,Open,High,Low,Vol.,Change %,j_quote,Price_unsplited
1148,2020-07-27,94.81,93.71,94.91,93.48,121210000.0,2.38%,1148,379.24
1149,2020-07-28,93.25,94.37,94.55,93.25,103630000.0,-1.65%,1149,373.00
1150,2020-07-29,95.04,93.75,95.23,93.71,90330000.0,1.92%,1150,380.16
1151,2020-07-30,96.19,94.19,96.30,93.77,158130000.0,1.21%,1151,384.76
1152,2020-07-31,106.26,102.88,106.42,100.83,374300000.0,10.47%,1152,425.04
1153,2020-08-03,108.94,108.20,111.64,107.89,308150000.0,2.52%,1153,435.76
1154,2020-08-04,109.67,109.13,110.79,108.39,172790000.0,0.67%,1154,438.68
1155,2020-08-05,110.06,109.38,110.39,108.90,121990000.0,0.36%,1155,440.24
1156,2020-08-06,113.90,110.41,114.41,109.80,202430000.0,3.49%,1156,455.60
1157,2020-08-07,111.11,113.20,113.67,110.29,198050000.0,-2.45%,1157,444.44


In [45]:
type(df_price.iloc[0])

pandas.core.series.Series

In [35]:
df_price.iloc[1152]

Date                2020-07-31
Price                   106.26
Open                    102.88
High                    106.42
Low                     100.83
Vol.               374300000.0
Change %                10.47%
j_quote                   1152
Price_unsplited         425.04
Name: 1152, dtype: object

In [23]:
df_price.loc[" 2020-07-31"]

KeyError: ' 2020-07-31'

In [25]:
df_train = pd.read_pickle("df_train.pkl")
df_validation = pd.read_pickle("df_validation.pkl")
df_test = pd.read_pickle("df_test.pkl")
df_price = pd.read_pickle("df_price.pkl")

In [22]:
df_price.columns

Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %', 'j_quote',
       'Price_unsplited'],
      dtype='object')